In [19]:
#!/usr/bin/env python
import requests
import json
import gzip
from tqdm import tqdm

In [25]:
if __name__ == '__main__':
    uniprot_pmid = {}  # { pmid : list_of_uniprot_ids }
    with open("res/uniprot_pubmed_id.tab") as f:
        for line in f:
            line = line.strip().split("\t")
            if len(line) == 3:
                for pmid in line[2].split("; "):
                    uniprot_pmid.setdefault(pmid, []).append(line[0])

    inverted = {}
    for k, v in uniprot_pmid.items():
        if v[0] != 'Entry':
            for prot in v:
                inverted.setdefault(prot, []).append(k)

    # Parse the disease ontology
    do = {}  # { do_id : do_object }
    obj = {}  # { id: do_id, name: definition, xref: list_of_omim_ids, is_a: list_of_parents, is_obsolete: True }
    with open("res/doid.obo") as f:
        for line in f:
            line = line.strip().split(": ")
            if line and len(line) == 2:
                k, v = line
                if k == "id" and v.startswith("DOID:"):
                    obj["id"] = v.split(":")[1]
                elif k == "xref" and "OMIM" in v:
                    obj["omim"] = v.split(":")[1]
                elif k == "name":
                    obj["name"] = v
                elif k == "is_a":
                    obj.setdefault("is_a", []).append(v.split()[0].split(":")[1])
                elif k == "is_obsolete":
                    obj["is_obsolete"] = True
            else:
                if obj.get("id") and not obj.get("is_obsolete"):
                    do[obj["id"]] = obj
                obj = {}

    name_do = {}
    for k, v in do.items():
        name_do.setdefault(v['name'], k)

    # *** How can you find all disease annotations associated to human PubMed abstracts?

    diseases = {}  # { uniprot_id : list_of_diseases }
    pmids = list(uniprot_pmid.keys())
    URL = "https://www.ebi.ac.uk/europepmc/annotations_api/annotationsByArticleIds"
    for i in tqdm(range(0, len(pmids), 8)):
        params = {"articleIds": ",".join(["MED:{}".format(pmid) for pmid in pmids[i:i + 8]]), "type": "Diseases",
                  "section"   : "Abstract", "format": "JSON"}
        r = requests.get(URL, params=params)
        obj = json.loads(r.text)
        for ele in obj:
            for annotation in ele.get("annotations"):
                for uniprot_id in uniprot_pmid[ele["extId"]]:
                    if annotation["exact"] in name_do.keys():
                        DOID = name_do[annotation["exact"]]
                        diseases.setdefault(uniprot_id, set()).add(DOID)
    diseases = {k: list(v) for k, v in diseases.items()}
    json = json.dumps(diseases)
    f = open("datasets/uniprot_do.json", "w")
    f.write(json)
    f.close()




  0%|          | 0/9601 [00:00<?, ?it/s]


  0%|          | 1/9601 [00:00<29:35,  5.41it/s]


  0%|          | 2/9601 [00:00<32:26,  4.93it/s]


  0%|          | 3/9601 [00:00<33:36,  4.76it/s]


  0%|          | 4/9601 [00:00<32:40,  4.89it/s]


  0%|          | 5/9601 [00:01<1:00:03,  2.66it/s]


  0%|          | 6/9601 [00:02<1:01:20,  2.61it/s]


  0%|          | 7/9601 [00:03<2:11:46,  1.21it/s]


  0%|          | 8/9601 [00:07<4:29:37,  1.69s/it]


  0%|          | 9/9601 [00:07<3:20:22,  1.25s/it]


  0%|          | 10/9601 [00:08<2:53:14,  1.08s/it]


  0%|          | 11/9601 [00:09<2:54:51,  1.09s/it]


  0%|          | 12/9601 [00:10<2:43:00,  1.02s/it]


  0%|          | 13/9601 [00:11<2:22:45,  1.12it/s]


  0%|          | 14/9601 [00:11<2:16:14,  1.17it/s]


  0%|          | 15/9601 [00:12<1:48:30,  1.47it/s]


  0%|          | 16/9601 [00:13<2:03:28,  1.29it/s]


  0%|          | 17/9601 [00:15<3:15:23,  1.22s/it]


  0%|          | 18/9601 [00:18<4:46:02,  1.79s/it]



  2%|▏         | 153/9601 [01:20<1:33:05,  1.69it/s]


  2%|▏         | 154/9601 [01:24<4:08:24,  1.58s/it]


  2%|▏         | 155/9601 [01:24<3:04:51,  1.17s/it]


  2%|▏         | 156/9601 [01:27<4:37:09,  1.76s/it]


  2%|▏         | 157/9601 [01:27<3:23:42,  1.29s/it]


  2%|▏         | 158/9601 [01:29<3:51:54,  1.47s/it]


  2%|▏         | 159/9601 [01:32<4:44:31,  1.81s/it]


  2%|▏         | 160/9601 [01:32<3:29:28,  1.33s/it]


  2%|▏         | 161/9601 [01:32<2:40:04,  1.02s/it]


  2%|▏         | 162/9601 [01:33<2:01:33,  1.29it/s]


  2%|▏         | 163/9601 [01:33<1:35:56,  1.64it/s]


  2%|▏         | 164/9601 [01:33<1:16:22,  2.06it/s]


  2%|▏         | 165/9601 [01:33<1:04:38,  2.43it/s]


  2%|▏         | 166/9601 [01:33<55:52,  2.81it/s]  


  2%|▏         | 167/9601 [01:35<1:45:13,  1.49it/s]


  2%|▏         | 168/9601 [01:35<1:25:05,  1.85it/s]


  2%|▏         | 169/9601 [01:36<1:23:23,  1.88it/s]


  2%|▏         | 170/9601 [01:36<1:11:35,  2.20it/s]


  2%|▏    

  3%|▎         | 304/9601 [02:37<4:19:20,  1.67s/it]


  3%|▎         | 305/9601 [02:39<4:19:34,  1.68s/it]


  3%|▎         | 306/9601 [02:39<3:12:09,  1.24s/it]


  3%|▎         | 307/9601 [02:39<2:24:16,  1.07it/s]


  3%|▎         | 308/9601 [02:39<1:51:34,  1.39it/s]


  3%|▎         | 309/9601 [02:40<1:29:01,  1.74it/s]


  3%|▎         | 310/9601 [02:40<1:15:47,  2.04it/s]


  3%|▎         | 311/9601 [02:41<1:50:28,  1.40it/s]


  3%|▎         | 312/9601 [02:41<1:28:11,  1.76it/s]


  3%|▎         | 313/9601 [02:42<1:43:10,  1.50it/s]


  3%|▎         | 314/9601 [02:42<1:23:03,  1.86it/s]


  3%|▎         | 315/9601 [02:43<1:09:53,  2.21it/s]


  3%|▎         | 316/9601 [02:46<3:04:56,  1.20s/it]


  3%|▎         | 317/9601 [02:46<2:23:16,  1.08it/s]


  3%|▎         | 318/9601 [02:46<1:49:20,  1.41it/s]


  3%|▎         | 319/9601 [02:47<1:48:38,  1.42it/s]


  3%|▎         | 320/9601 [02:47<1:26:07,  1.80it/s]


  3%|▎         | 321/9601 [02:48<1:38:04,  1.58it/s]


  3%|▎    

  5%|▍         | 454/9601 [04:03<1:17:35,  1.96it/s]


  5%|▍         | 455/9601 [04:05<2:15:54,  1.12it/s]


  5%|▍         | 456/9601 [04:05<1:46:14,  1.43it/s]


  5%|▍         | 457/9601 [04:05<1:22:25,  1.85it/s]


  5%|▍         | 458/9601 [04:05<1:06:04,  2.31it/s]


  5%|▍         | 459/9601 [04:05<55:18,  2.76it/s]  


  5%|▍         | 460/9601 [04:06<1:01:01,  2.50it/s]


  5%|▍         | 461/9601 [04:07<1:33:41,  1.63it/s]


  5%|▍         | 462/9601 [04:08<1:39:56,  1.52it/s]


  5%|▍         | 463/9601 [04:10<3:06:25,  1.22s/it]


  5%|▍         | 464/9601 [04:12<3:27:34,  1.36s/it]


  5%|▍         | 465/9601 [04:12<2:42:33,  1.07s/it]


  5%|▍         | 466/9601 [04:13<2:04:13,  1.23it/s]


  5%|▍         | 467/9601 [04:13<1:45:27,  1.44it/s]


  5%|▍         | 468/9601 [04:13<1:28:57,  1.71it/s]


  5%|▍         | 469/9601 [04:14<1:29:37,  1.70it/s]


  5%|▍         | 470/9601 [04:14<1:14:31,  2.04it/s]


  5%|▍         | 471/9601 [04:14<1:02:14,  2.44it/s]


  5%|▍    

  6%|▋         | 605/9601 [05:18<2:29:57,  1.00s/it]


  6%|▋         | 606/9601 [05:19<2:38:54,  1.06s/it]


  6%|▋         | 607/9601 [05:21<3:12:10,  1.28s/it]


  6%|▋         | 608/9601 [05:21<2:22:42,  1.05it/s]


  6%|▋         | 609/9601 [05:21<1:52:03,  1.34it/s]


  6%|▋         | 610/9601 [05:22<1:27:57,  1.70it/s]


  6%|▋         | 611/9601 [05:22<1:11:48,  2.09it/s]


  6%|▋         | 612/9601 [05:22<1:07:07,  2.23it/s]


  6%|▋         | 613/9601 [05:22<1:00:36,  2.47it/s]


  6%|▋         | 614/9601 [05:23<54:35,  2.74it/s]  


  6%|▋         | 615/9601 [05:23<51:00,  2.94it/s]


  6%|▋         | 616/9601 [05:24<1:04:16,  2.33it/s]


  6%|▋         | 617/9601 [05:24<58:36,  2.55it/s]  


  6%|▋         | 618/9601 [05:24<51:17,  2.92it/s]


  6%|▋         | 619/9601 [05:24<46:21,  3.23it/s]


  6%|▋         | 620/9601 [05:25<44:03,  3.40it/s]


  6%|▋         | 621/9601 [05:25<46:02,  3.25it/s]


  6%|▋         | 622/9601 [05:25<48:30,  3.09it/s]


  6%|▋         | 623/9

...

 92%|█████████▏| 8869/9601 [58:24<02:26,  5.00it/s]


 92%|█████████▏| 8870/9601 [58:24<02:37,  4.65it/s]


 92%|█████████▏| 8871/9601 [58:24<02:28,  4.91it/s]


 92%|█████████▏| 8872/9601 [58:24<02:28,  4.90it/s]


 92%|█████████▏| 8873/9601 [58:25<02:25,  5.01it/s]


 92%|█████████▏| 8874/9601 [58:25<02:40,  4.52it/s]


 92%|█████████▏| 8875/9601 [58:25<02:46,  4.36it/s]


 92%|█████████▏| 8876/9601 [58:25<02:52,  4.21it/s]


 92%|█████████▏| 8877/9601 [58:26<02:51,  4.22it/s]


 92%|█████████▏| 8878/9601 [58:26<02:45,  4.38it/s]


 92%|█████████▏| 8879/9601 [58:27<04:36,  2.61it/s]


 92%|█████████▏| 8880/9601 [58:27<03:56,  3.05it/s]


 93%|█████████▎| 8881/9601 [58:27<03:28,  3.45it/s]


 93%|█████████▎| 8882/9601 [58:27<03:10,  3.78it/s]


 93%|█████████▎| 8883/9601 [58:27<03:08,  3.81it/s]


 93%|█████████▎| 8884/9601 [58:28<02:57,  4.03it/s]


 93%|█████████▎| 8885/9601 [58:28<02:54,  4.09it/s]


 93%|█████████▎| 8886/9601 [58:28<02:39,  4.48it/s]


 93%|█████████▎| 8887/9601 [

 94%|█████████▍| 9020/9601 [58:58<02:19,  4.17it/s]


 94%|█████████▍| 9021/9601 [58:59<02:21,  4.09it/s]


 94%|█████████▍| 9022/9601 [58:59<02:19,  4.16it/s]


 94%|█████████▍| 9023/9601 [58:59<02:16,  4.23it/s]


 94%|█████████▍| 9024/9601 [58:59<02:15,  4.25it/s]


 94%|█████████▍| 9025/9601 [59:00<02:13,  4.31it/s]


 94%|█████████▍| 9026/9601 [59:00<03:38,  2.63it/s]


 94%|█████████▍| 9027/9601 [59:00<03:04,  3.10it/s]


 94%|█████████▍| 9028/9601 [59:01<02:47,  3.43it/s]


 94%|█████████▍| 9029/9601 [59:01<02:34,  3.70it/s]


 94%|█████████▍| 9030/9601 [59:01<02:20,  4.07it/s]


 94%|█████████▍| 9031/9601 [59:01<02:14,  4.23it/s]


 94%|█████████▍| 9032/9601 [59:02<02:10,  4.37it/s]


 94%|█████████▍| 9033/9601 [59:02<02:57,  3.20it/s]


 94%|█████████▍| 9034/9601 [59:02<02:40,  3.53it/s]


 94%|█████████▍| 9035/9601 [59:02<02:26,  3.86it/s]


 94%|█████████▍| 9036/9601 [59:03<02:15,  4.17it/s]


 94%|█████████▍| 9037/9601 [59:03<02:05,  4.50it/s]


 94%|█████████▍| 9038/9601 [

 96%|█████████▌| 9171/9601 [59:35<01:42,  4.21it/s]


 96%|█████████▌| 9172/9601 [59:35<01:37,  4.42it/s]


 96%|█████████▌| 9173/9601 [59:35<01:46,  4.01it/s]


 96%|█████████▌| 9174/9601 [59:35<01:39,  4.29it/s]


 96%|█████████▌| 9175/9601 [59:36<01:39,  4.27it/s]


 96%|█████████▌| 9176/9601 [59:36<01:35,  4.47it/s]


 96%|█████████▌| 9177/9601 [59:36<01:33,  4.52it/s]


 96%|█████████▌| 9178/9601 [59:36<01:29,  4.73it/s]


 96%|█████████▌| 9179/9601 [59:37<01:53,  3.72it/s]


 96%|█████████▌| 9180/9601 [59:37<01:48,  3.87it/s]


 96%|█████████▌| 9181/9601 [59:37<01:40,  4.17it/s]


 96%|█████████▌| 9182/9601 [59:37<01:39,  4.23it/s]


 96%|█████████▌| 9183/9601 [59:38<01:42,  4.08it/s]


 96%|█████████▌| 9184/9601 [59:38<01:38,  4.21it/s]


 96%|█████████▌| 9185/9601 [59:38<01:39,  4.19it/s]


 96%|█████████▌| 9186/9601 [59:38<01:33,  4.45it/s]


 96%|█████████▌| 9187/9601 [59:38<01:30,  4.55it/s]


 96%|█████████▌| 9188/9601 [59:39<01:32,  4.46it/s]


 96%|█████████▌| 9189/9601 [

 97%|█████████▋| 9321/9601 [1:00:10<01:08,  4.11it/s]


 97%|█████████▋| 9322/9601 [1:00:10<01:05,  4.25it/s]


 97%|█████████▋| 9323/9601 [1:00:10<01:08,  4.06it/s]


 97%|█████████▋| 9324/9601 [1:00:10<01:05,  4.22it/s]


 97%|█████████▋| 9325/9601 [1:00:11<01:01,  4.48it/s]


 97%|█████████▋| 9326/9601 [1:00:11<00:59,  4.59it/s]


 97%|█████████▋| 9327/9601 [1:00:11<01:00,  4.56it/s]


 97%|█████████▋| 9328/9601 [1:00:11<00:58,  4.71it/s]


 97%|█████████▋| 9329/9601 [1:00:12<01:03,  4.27it/s]


 97%|█████████▋| 9330/9601 [1:00:12<01:09,  3.91it/s]


 97%|█████████▋| 9331/9601 [1:00:12<01:24,  3.18it/s]


 97%|█████████▋| 9332/9601 [1:00:12<01:17,  3.49it/s]


 97%|█████████▋| 9333/9601 [1:00:13<01:10,  3.78it/s]


 97%|█████████▋| 9334/9601 [1:00:13<01:06,  4.01it/s]


 97%|█████████▋| 9335/9601 [1:00:13<01:02,  4.25it/s]


 97%|█████████▋| 9336/9601 [1:00:13<01:01,  4.32it/s]


 97%|█████████▋| 9337/9601 [1:00:14<00:58,  4.54it/s]


 97%|█████████▋| 9338/9601 [1:00:14<00:57,  4.59

 99%|█████████▊| 9467/9601 [1:00:43<00:31,  4.25it/s]


 99%|█████████▊| 9468/9601 [1:00:43<00:31,  4.18it/s]


 99%|█████████▊| 9469/9601 [1:00:44<00:40,  3.22it/s]


 99%|█████████▊| 9470/9601 [1:00:44<00:37,  3.53it/s]


 99%|█████████▊| 9471/9601 [1:00:44<00:33,  3.90it/s]


 99%|█████████▊| 9472/9601 [1:00:44<00:32,  3.92it/s]


 99%|█████████▊| 9473/9601 [1:00:44<00:29,  4.28it/s]


 99%|█████████▊| 9474/9601 [1:00:45<00:28,  4.52it/s]


 99%|█████████▊| 9475/9601 [1:00:45<00:26,  4.70it/s]


 99%|█████████▊| 9476/9601 [1:00:45<00:25,  4.88it/s]


 99%|█████████▊| 9477/9601 [1:00:45<00:25,  4.84it/s]


 99%|█████████▊| 9478/9601 [1:00:46<00:29,  4.18it/s]


 99%|█████████▊| 9479/9601 [1:00:46<00:29,  4.16it/s]


 99%|█████████▊| 9480/9601 [1:00:46<00:27,  4.40it/s]


 99%|█████████▉| 9481/9601 [1:00:46<00:25,  4.76it/s]


 99%|█████████▉| 9482/9601 [1:00:46<00:25,  4.70it/s]


 99%|█████████▉| 9483/9601 [1:00:47<00:25,  4.70it/s]


 99%|█████████▉| 9484/9601 [1:00:47<00:24,  4.73